In [1]:
import wget
import pandas as pd
import gzip
import shutil
import os

In [2]:
colspecs = [(0, 11), (13, 20), (22, 30), (32, 37), (38, 40), (41, -1)]
stations_df = pd.read_fwf('../data/lookup_data/station_state_city.txt', colspecs=colspecs, names=['station_id', 'latitude', 'longitude', 'elevation', 'state_code', 'station_name'])
stations_df.head(10)

,station_id,latitude,longitude,elevation,state_code,station_name
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA)
1,US10RMHS145,40.5268,105.1113,569.1,CO,RMHS 1.6 SSW
2,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S
3,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW
4,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW
5,US10adam004,40.4798,-98.4026,570.0,NE,AYR 3.5 NE
6,US10adam006,40.4372,-98.5912,601.1,NE,ROSELAND 2.8 SW
7,US10adam007,40.5389,-98.4713,588.9,NE,HASTINGS 5.4 WSW
8,US10adam008,40.4953,-98.2973,566.9,NE,GLENVIL 2.3 WSW
9,US10adam010,40.5532,-98.6297,622.1,NE,JUNIATA 6.9 WSW


In [4]:
# for i in range(118):
#     yr = i + 1905
for i in range(114):
    yr = i + 1905
    print(f"processing {yr}")
    remote_url = f'https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/{yr}.csv.gz'
    if os.path.exists('local_copy.csv.gz'):
        os.remove('local_copy.csv.gz')
    if os.path.exists('local_copy.csv'):
        os.remove('local_copy.csv')    
    wget.download(remote_url, 'local_copy.csv.gz')
    #shcopyfile = f'/Users/sanilveeravu/Downloads/archive-2/ghcnd_all_years/{yr}.csv.gz'
    #shutil.copyfile(shcopyfile, 'local_copy.csv.gz')
    print("Copy Complete")
    with gzip.open('local_copy.csv.gz', 'rb') as f_in:
        with open('local_copy.csv', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print("Unzip Complete")        
    df = pd.read_csv('local_copy.csv', names=["station_id", "date", "metric", "value", "measurement_flag", "quality_flag", "source_flag", "observation_time"] )
    df = df[df.station_id.str[:2] == "US"]
    df = df[(df.metric == "SNOW")|(df.metric == "PRCP")|(df.metric == "TMIN")|(df.metric == "TMAX")]
    print("Filtering Complete")
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
#     df['year'] = pd.DatetimeIndex(df['date']).year
#     df['month'] = pd.DatetimeIndex(df['date']).month
#     df['day'] = pd.DatetimeIndex(df['date']).day
    df = df[df.quality_flag.isnull()]
    print("Cleanup Complete")
    df = pd.merge(df, stations_df, how='inner', on = 'station_id')
    df['value'] = df.apply(lambda x: x['value'] if x['metric']=="SNOW" else x['value']/10, axis=1)
    df.drop(["station_id","station_name","measurement_flag","quality_flag","source_flag","observation_time","latitude","longitude","elevation"], axis=1, inplace=True)
    print("Conversion Complete")
    tmindata = df[df.metric=="TMIN"].groupby(["state_code", "date"]).agg("min").reset_index()
    tmaxdata = df[df.metric=="TMAX"].groupby(["state_code", "date"]).agg("max").reset_index()
    prcpdata = df[df.metric=="PRCP"].groupby(["state_code", "date"]).agg("max").reset_index()
    snowdata = df[df.metric=="SNOW"].groupby(["state_code", "date"]).agg("max").reset_index()
    tmindatata.drop(["metric"], axis=1, inplace=True)
    tmaxdata.drop(["metric"], axis=1, inplace=True)
    prcpdata.drop(["metric"], axis=1, inplace=True)
    snowdata.drop(["metric"], axis=1, inplace=True)
    print("Aggregation Complete")
    tmindata.to_csv('../data/cleaned_data/tmindata.csv', mode='a', index=False, header=False)
    tmaxdata.to_csv('../data/cleaned_data/tmaxdata.csv', mode='a', index=False, header=False)
    prcpdata.to_csv('../data/cleaned_data/prcpdata.csv', mode='a', index=False, header=False)
    snowdata.to_csv('../data/cleaned_data/snowdata.csv', mode='a', index=False, header=False)

processing 1905
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1906
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1907
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1908
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1909
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1910
Copy Complete
Unzip Complete


/Users/sanilveeravu/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1911
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1912
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1913
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1914
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1915
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1916
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1917
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1918
Copy Complete
Unzip Complete
Filtering Complete
Clean

Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1978
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1979
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1980
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1981
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1982
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1983
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1984
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Conversion Complete
Aggregation Complete
processing 1985
Copy Complete
Unzip Complete
Filtering Complete
Cleanup Complete
Convers